# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
import sys
sys.path.append("../../") # root of all code is 2 floors below

In [ ]:
from unityagents import UnityEnvironment, UnityEnvironmentException
import numpy as np
from tqdm import tqdm

from model import QNetwork
import random
import torch
import numpy as np
from torch import nn
from collections import deque
import matplotlib.pyplot as plt
import time
%matplotlib inline
%load_ext autoreload
%autoreload 2


Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [ ]:
# env = UnityEnvironment(file_name="Banana.app")
try:
    env = UnityEnvironment(file_name="Banana_Linux_NoVis/Banana.x86_64") # Linux
    print("we are on Linux")
except UnityEnvironmentException:
    # try Mac
    env = UnityEnvironment(file_name="Banana.app")
    print("we are on Mac")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)


In [ ]:
import torch
torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [ ]:
# create an agent
from dqn_agent import DQNAgent, DoubleDQNAgent
from memory import ReplayBuffer, WeightedReplayBuffer

hidden_sizes = [int(round(state_size * .8)), int(round(state_size * .6)), int(round(action_size * 2))]
# self.hidden_sizes = [int(round(state_size * 10)), int(round(state_size * 5)), int(round(action_size * 10))]
fc = nn.Sequential(
    nn.Linear(state_size, hidden_sizes[0]),
    nn.ReLU(),
    nn.Linear(hidden_sizes[0], hidden_sizes[1]),
    nn.ReLU(),
    nn.Linear(hidden_sizes[1], hidden_sizes[2]),
    nn.ReLU(),
    nn.Linear(hidden_sizes[2], action_size)
)
fc2 = nn.Sequential(
    nn.Linear(state_size, hidden_sizes[0]),
    nn.ReLU(),
    nn.Linear(hidden_sizes[0], hidden_sizes[1]),
    nn.ReLU(),
    nn.Linear(hidden_sizes[1], hidden_sizes[2]),
    nn.ReLU(),
    nn.Linear(hidden_sizes[2], action_size)
)

main_model = QNetwork(name="main", fc=fc)
target_model = QNetwork(name="target", fc=fc2)


In [ ]:
# #################################
# this agent solves the problem!
solving_agent = DQNAgent(
    main_model=main_model,
    target_network=target_model,
    update_every_steps=10, 
    lr=1e-3)
# #################################

agent_dqn = DQNAgent(
    main_model=main_model,
    target_network=target_model,
    update_every_steps=10, 
    lr=1e-3)

agent_dqn_prioritized_replay = DQNAgent(
    main_model=main_model,
    target_network=target_model,
    memory = WeightedReplayBuffer(buffer_size=int(1e5), batch_size=64),
    update_every_steps=10, 
    lr=1e-3)


In [ ]:
# agent_double_dqn = DoubleDQNAgent(
#    main_model=main_model,
#    target_network=target_model,
#    is_double=True,
#    update_every_steps=10, 
#    lr=1e-3, 
#    batch_size=64, 
#    memory_size=int(1e5))

In [ ]:
from runner.unity import run as UnityRun

prob_not_random_at_start_default=(1-0.995) # /15
scores_dqn = UnityRun(
        solving_agent,
        env, brain_name,
        n_episodes=1000, 
    feedback_every_secs=45, 
    keep_last_scores=100, 
    eps_decay=1 - prob_not_random_at_start_default)

In [ ]:
# agent.set_models_from(torch_file_name='checkpoint.pth')
prob_not_random_at_start_default=(1-0.995) # /15
scores_dqn = dqn(
    solving_agent, 
    n_episodes=20, 
    feedback_every_secs=45, 
    keep_last_scores=100, 
    eps_decay=1 - prob_not_random_at_start_default)

In [ ]:
prob_not_random_at_start_default=(1-0.995) # /15
scores_double_dqn = dqn(
    agent_double_dqn, 
    n_episodes=1000, 
    feedback_every_secs=45, 
    keep_last_scores=100, 
    eps_decay=1 - prob_not_random_at_start_default)

In [ ]:
1 - (1-0.995)/25

In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores_double_dqn)), scores_double_dqn)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.title('Double DQN')
plt.show()

In [ ]:
# scores = dqn(agent, n_episodes=50, feedback_every_secs=15, keep_last_scores=10)

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores_dqn)), scores_dqn)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.title('DQN')
plt.show()

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```